# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company **Drop** 
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned **Drop**, because randomly assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded **Drop** 
    Location: Client HQ Address **Drop** 
    Company: Name of Client Company **Drop** 
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('bin_customer').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

### Retain columns of value

In [7]:
final_data = data.drop('Names','Account_Manager','Onboard_date','Location','Company')
final_data.show()

+----+--------------+-----+---------+-----+
| Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
|42.0|       11066.8| 7.22|      8.0|    1|
|41.0|      11916.22|  6.5|     11.0|    1|
|38.0|      12884.75| 6.67|     12.0|    1|
|42.0|       8010.76| 6.71|     10.0|    1|
|37.0|       9191.58| 5.56|      9.0|    1|
|48.0|      10356.02| 5.12|      8.0|    1|
|44.0|      11331.58| 5.23|     11.0|    1|
|32.0|       9885.12| 6.92|      9.0|    1|
|43.0|       14062.6| 5.46|     11.0|    1|
|40.0|       8066.94| 7.11|     11.0|    1|
|30.0|      11575.37| 5.22|      8.0|    1|
|45.0|       8771.02| 6.64|     11.0|    1|
|45.0|       8988.67| 4.84|     11.0|    1|
|40.0|       8283.32|  5.1|     13.0|    1|
|41.0|       6569.87|  4.3|     11.0|    1|
|38.0|      10494.82| 6.81|     12.0|    1|
|45.0|       8213.41| 7.35|     11.0|    1|
|43.0|      11226.88| 8.08|     12.0|    1|
|53.0|       5515.09| 6.85|      8.0|    1|
|46.0|        8046.4| 5.69|     

### Collaspe data into a vector

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
final_data = assembler.transform(final_data)

In [11]:
final_data.select('features','Churn').show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,7.2...|    1|
|[41.0,11916.22,6....|    1|
|[38.0,12884.75,6....|    1|
|[42.0,8010.76,6.7...|    1|
|[37.0,9191.58,5.5...|    1|
|[48.0,10356.02,5....|    1|
|[44.0,11331.58,5....|    1|
|[32.0,9885.12,6.9...|    1|
|[43.0,14062.6,5.4...|    1|
|[40.0,8066.94,7.1...|    1|
|[30.0,11575.37,5....|    1|
|[45.0,8771.02,6.6...|    1|
|[45.0,8988.67,4.8...|    1|
|[40.0,8283.32,5.1...|    1|
|[41.0,6569.87,4.3...|    1|
|[38.0,10494.82,6....|    1|
|[45.0,8213.41,7.3...|    1|
|[43.0,11226.88,8....|    1|
|[53.0,5515.09,6.8...|    1|
|[46.0,8046.4,5.69...|    1|
+--------------------+-----+
only showing top 20 rows



In [12]:
train, test = final_data.randomSplit([0.7,0.3])

### Logistic Regression

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
lr = LogisticRegression(labelCol='Churn')

In [15]:
final_data.show()

+----+--------------+-----+---------+-----+--------------------+
| Age|Total_Purchase|Years|Num_Sites|Churn|            features|
+----+--------------+-----+---------+-----+--------------------+
|42.0|       11066.8| 7.22|      8.0|    1|[42.0,11066.8,7.2...|
|41.0|      11916.22|  6.5|     11.0|    1|[41.0,11916.22,6....|
|38.0|      12884.75| 6.67|     12.0|    1|[38.0,12884.75,6....|
|42.0|       8010.76| 6.71|     10.0|    1|[42.0,8010.76,6.7...|
|37.0|       9191.58| 5.56|      9.0|    1|[37.0,9191.58,5.5...|
|48.0|      10356.02| 5.12|      8.0|    1|[48.0,10356.02,5....|
|44.0|      11331.58| 5.23|     11.0|    1|[44.0,11331.58,5....|
|32.0|       9885.12| 6.92|      9.0|    1|[32.0,9885.12,6.9...|
|43.0|       14062.6| 5.46|     11.0|    1|[43.0,14062.6,5.4...|
|40.0|       8066.94| 7.11|     11.0|    1|[40.0,8066.94,7.1...|
|30.0|      11575.37| 5.22|      8.0|    1|[30.0,11575.37,5....|
|45.0|       8771.02| 6.64|     11.0|    1|[45.0,8771.02,6.6...|
|45.0|       8988.67| 4.8

In [16]:
classifier = lr.fit(train)

In [17]:
# more like a training summary
classifier.summary.predictions.describe().show()

+-------+-----------------+------------------+-----------------+------------------+-------------------+-------------------+
|summary|              Age|    Total_Purchase|            Years|         Num_Sites|              Churn|         prediction|
+-------+-----------------+------------------+-----------------+------------------+-------------------+-------------------+
|  count|              616|               616|              616|               616|                616|                616|
|   mean|41.94318181818182|10063.379366883108|5.254983766233768| 8.587662337662337|0.16883116883116883|0.12337662337662338|
| stddev|6.176932088195008| 2445.943295640931|1.296941241961982|1.7665667560794105| 0.3749071612809897|0.32913628438163306|
|    min|             26.0|             100.0|              1.0|               3.0|                0.0|                0.0|
|    max|             65.0|          18026.01|             9.15|              14.0|                1.0|                1.0|
+-------

In [18]:
prediction = classifier.evaluate(test)

In [19]:
prediction = prediction.predictions.select('Churn','prediction')

In [20]:
prediction.show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



### Evaluator 

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [22]:
# for two classes like 0/1
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [23]:
# For multiclass
# evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Churn',
#                                             metricName='accuracy')

In [24]:
# ROC = Receiver Operating Characteristic
auc = evaluator.evaluate(prediction)
auc

0.7984106686152723

## Predict on new data

In [25]:
final_lr_model = lr.fit(final_data)

In [26]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [27]:
# same as before
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [28]:
test_new_customers = assembler.transform(new_customers)

In [29]:
# look for features column at the end
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [30]:
final_results = final_lr_model.transform(test_new_customers)

In [31]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [32]:
# there was only 6 customers to predict
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                null|            null|
|    min|Andrew Mccall|              22.0|            100.0|          